In [2]:
import json
import numpy as np
from torchvision import datasets, transforms

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick to prevent overflow
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

# Global storage (simulating Redis)
storage = {}

# Dictionary to hold function mappings
function_map = {}  # { "layer_0": ["neuron_0_0", "neuron_0_1", ...], "layer_1": [...] }

# Function registry (will hold dynamically created functions)
function_registry = {}

# Function to create neuron functions dynamically
def create_neuron_function(weights, bias, activation, layer_id, neuron_id, is_final_layer):
    def neuron():
        # Determine input keys
        prev_layer = f"layer_{int(layer_id.split('_')[-1]) - 1}" if layer_id != "layer_0" else None
        input_keys = [f"input_{i}" for i in range(len(weights))] if prev_layer is None else \
                     [f"{prev_layer}_{j}" for j in range(len(weights))]

        # Fetch input values from storage
        input_values = np.array([storage.get(inp, 0) for inp in input_keys])

        if input_values.shape[0] != len(weights):
            raise ValueError(f"{layer_id}_{neuron_id} expects {len(weights)} inputs but got {input_values.shape[0]}.")

        # Compute activation
        z = np.dot(weights, input_values) + bias
        output = z if is_final_layer else ACTIVATIONS[activation](z)

        # Store output
        storage[f"{layer_id}_{neuron_id}"] = output
        return output

    function_registry[f"{layer_id}_{neuron_id}"] = neuron  # Register function

# Function to create layer functions dynamically
def create_layer_function(layer_id, neuron_ids, is_final_layer):
    def layer():
        neuron_outputs = [function_registry[neuron]() for neuron in neuron_ids]
        
        # If it's the final layer, apply softmax
        if is_final_layer:
            softmax_outputs = softmax(np.array(neuron_outputs))
            for i, val in enumerate(softmax_outputs):
                storage[f"{layer_id}_{i}"] = val

    function_registry[layer_id] = layer  # Register function

# Build network as functions
def build_network(json_data):
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))

    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neuron_ids = []

        for j, node in enumerate(layer_info['nodes']):
            neuron_id = f"{layer_name}_{j}"
            neuron_ids.append(neuron_id)

            create_neuron_function(
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                layer_id=layer_name,
                neuron_id=j,
                is_final_layer=(i == len(sorted_layers) - 1)
            )

        function_map[layer_name] = neuron_ids  # Store neurons of this layer
        create_layer_function(layer_name, neuron_ids, is_final_layer=(i == len(sorted_layers) - 1))

# Forward pass using function calls
def forward_pass(layers, input_data):
    storage.clear()  # Clear storage before processing new image

    # Initialize input storage
    for i, val in enumerate(input_data):
        storage[f"input_{i}"] = val

    # Call each layer function in sequence
    for layer in layers:
        function_registry[layer]()  # Call layer function

    # Retrieve final layer outputs
    final_layer = layers[-1]
    final_outputs = np.array([storage[f"{final_layer}_{j}"] for j in range(len(function_map[final_layer]))])

    return np.argmax(final_outputs)  # Return predicted class

# Load network
data = load_network("node_based_model.json")
build_network(data)

# Load MNIST dataset with correct normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test on MNIST images
for i in range(10):  # Testing on first 10 images
    image, label = mnist_test[i]
    image = image.view(-1).numpy()  # Flatten 28x28 image to a 784 input vector
    
    predicted_class = forward_pass(list(function_map.keys()), image)
    print(f"Image {i}: Label = {label}, Predicted = {predicted_class}")


Image 0: Label = 7, Predicted = 7
Image 1: Label = 2, Predicted = 2
Image 2: Label = 1, Predicted = 1
Image 3: Label = 0, Predicted = 0
Image 4: Label = 4, Predicted = 4
Image 5: Label = 1, Predicted = 1
Image 6: Label = 4, Predicted = 4
Image 7: Label = 9, Predicted = 9
Image 8: Label = 5, Predicted = 6
Image 9: Label = 9, Predicted = 9
